## Patient Type Prediction

Given data about hospital patients, let's try to predict if a given patient will be **an outpatient or an impatient**

Data Source: https://www.kaggle.com/datasets/saurabhshahane/patient-treatment-classification?select=data-ori.csv

### Getting Started

In [14]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
data = pd.read_csv("data-ori.csv")
data

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SEX,SOURCE
0,35.1,11.8,4.65,6.3,310,25.4,33.6,75.5,1,F,out
1,43.5,14.8,5.39,12.7,334,27.5,34.0,80.7,1,F,out
2,33.5,11.3,4.74,13.2,305,23.8,33.7,70.7,1,F,out
3,39.1,13.7,4.98,10.5,366,27.5,35.0,78.5,1,F,out
4,30.9,9.9,4.23,22.1,333,23.4,32.0,73.0,1,M,out
...,...,...,...,...,...,...,...,...,...,...,...
4407,32.8,10.4,3.49,8.1,72,29.8,31.7,94.0,92,F,in
4408,33.7,10.8,3.67,6.7,70,29.4,32.0,91.8,92,F,in
4409,33.2,11.2,3.47,7.2,235,32.3,33.7,95.7,93,F,out
4410,31.5,10.4,3.15,9.1,187,33.0,33.0,100.0,98,F,in


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4412 entries, 0 to 4411
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HAEMATOCRIT   4412 non-null   float64
 1   HAEMOGLOBINS  4412 non-null   float64
 2   ERYTHROCYTE   4412 non-null   float64
 3   LEUCOCYTE     4412 non-null   float64
 4   THROMBOCYTE   4412 non-null   int64  
 5   MCH           4412 non-null   float64
 6   MCHC          4412 non-null   float64
 7   MCV           4412 non-null   float64
 8   AGE           4412 non-null   int64  
 9   SEX           4412 non-null   object 
 10  SOURCE        4412 non-null   object 
dtypes: float64(7), int64(2), object(2)
memory usage: 379.3+ KB


### Preprocessing

In [8]:
def preprocess_inputs(df):
    df = df.copy()
    # Binary encoding
    df['SEX'] = df['SEX'].replace({'F':0, 'M': 1})

    # Split df into X and y
    y = df['SOURCE']
    X = df.drop('SOURCE', axis=1)

    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

/tmp/ipykernel_1571/3699909917.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['SEX'] = df['SEX'].replace({'F':0, 'M': 1})


In [10]:
X_train

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SEX
2275,1.521571,0.918324,4.205082,-0.507635,-0.368181,-3.543672,-1.886871,-3.523787,0.113088,-1.043023
4093,-0.590225,-0.613524,-0.673427,-0.468456,-0.184678,0.106741,-0.267275,0.263954,1.411455,-1.043023
1727,-1.512041,-1.618799,-1.463160,0.099634,1.195966,-0.228501,-0.996093,0.191392,-0.350615,-1.043023
615,0.817639,0.870454,0.765924,-0.488046,-0.140986,0.032243,0.380563,-0.127881,-1.138910,0.958752
1610,0.348351,0.391752,-0.036546,-0.311742,0.260973,0.665478,0.299583,0.670302,-0.443356,-1.043023
...,...,...,...,...,...,...,...,...,...,...
2895,0.029906,0.056660,-0.240348,-0.781885,0.531859,0.479233,0.137624,0.510665,0.576790,-1.043023
2763,-0.003615,-0.278432,-0.533314,-0.703528,0.182329,0.479233,-1.158053,1.149211,0.484050,-1.043023
905,1.320448,1.444897,0.753186,-0.488046,0.907604,0.926222,0.704482,0.742864,-0.953429,0.958752
3980,-0.539944,-0.565653,-0.558789,-0.703528,-0.420611,-0.005006,-0.267275,0.089805,1.318715,0.958752


In [12]:
y_train.value_counts()

SOURCE
out    1834
in     1254
Name: count, dtype: int64

### Training

In [15]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Neural Network": MLPClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

Logistic Regression trained.
Decision Tree trained.
Neural Network trained.
Random Forest trained.
Gradient Boosting trained.


### Results

In [16]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(name + " Accuracy: {:.2f}%".format(acc*100))

Logistic Regression Accuracy: 71.22%
Decision Tree Accuracy: 66.16%
Neural Network Accuracy: 74.09%
Random Forest Accuracy: 74.09%
Gradient Boosting Accuracy: 73.72%


In [18]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, pos_label='in')
    print(name + " F1-Score: {:.5f}".format(f1))

Logistic Regression F1-Score: 0.59768
Decision Tree F1-Score: 0.58899
Neural Network F1-Score: 0.66987
Random Forest F1-Score: 0.66471
Gradient Boosting F1-Score: 0.64848
